In [5]:
import math
import numpy as np
import pandas as pd
import yfinance as yf
from pypfopt.efficient_frontier import EfficientFrontier
from func import *
pd.DataFrame.iteritems = pd.DataFrame.items

In [11]:
portfolio  = ['AAPL',	'MSFT',	'AMZN',	'TSLA', 'BABA']
n_samples = 500
path = 'dataAdjClose.csv'
max_len_portfolio = 10
type_prize = 'Close' #Open	High	Low	Close
period = '5y'

In [12]:
def suboptimal_investment_portfolio(portfolio):
  suboptimal_portfolio = []

  # Iteramos y asignamos proporción igualitaria de inversión a cada
  # activo del portafolio.
  for x in portfolio:
    suboptimal_portfolio.append(1/len(portfolio))

  suboptimal_portfolio = np.array(suboptimal_portfolio)
  return suboptimal_portfolio

In [15]:

if len(portfolio) > max_len_portfolio:
  raise Exception (f'El número máximo de activos en el portafolio es de {max_len_portfolio}')
else:
    stock_prices = pd.DataFrame()
    stock_returns = pd.DataFrame()

    for x in portfolio:
        stock = yf.Ticker(x)
        close_price = stock.history(period=period)[type_prize]

        # Se insertan datos en stock_prices y stock_returns
        stock_prices = pd.concat([stock_prices, close_price], axis=1)
        # Con pct_change() obtenemos los rendimientos (cambio porcentual)
        stock_returns = pd.concat([stock_returns, close_price.pct_change()], axis=1)

# Asignamos nombre de acciones a las columnas de cada DataFrame

stock_prices.columns = portfolio
stock_returns.columns = portfolio

# Eliminamos valores nulos de las columnas con dropna()
# (El primer valor de rendimiento es nulo)

stock_returns = stock_returns.dropna()

# # Ver los datos generados:
# print('Precios diarios, últimos 5 años (máximo)')
# display(stock_prices)
# print('Rendimientos diarios, últimos 5 años (máximo)')
# display(stock_returns)

# Rendimiento esperado de activos
expected_stock_returns = []

# Riesgo individual de activos
individual_stock_risk = []

for x, y in stock_returns.iteritems():
  # En cada iteración se obtiene el rendimiento esperado y riesgo individual
  # de cada activo (media y desviación estándar)
  expected_stock_returns.append(y.mean())
  individual_stock_risk.append(y.std())

stock_returns_cov_matrix = np.array(stock_returns.cov())

### OPTIMIZACION DE MARKOWITZ ###
ef = EfficientFrontier(expected_stock_returns, stock_returns_cov_matrix, weight_bounds=(0,1))
ratios = ef.min_volatility()
cleaned_ratios = pd.Series(ratios)
cleaned_ratios.index = portfolio

optimal_portfolio = np.expand_dims(cleaned_ratios, axis=0)

# Rendimiento esperado
opt_portfolio_expected_return = np.matmul(optimal_portfolio, expected_stock_returns)

# Varianza del portafolio
opt_portfolio_var = np.matmul(optimal_portfolio, \
                              np.matmul(stock_returns_cov_matrix, optimal_portfolio.transpose()))

# Riesgo del portafolio
opt_portfolio_risk = math.sqrt(opt_portfolio_var)

# Ver datos obtenidos y Portafolio Óptimo de Markowitz:
print('PORTAFOLIO ÓPTIMO DE MARKOWITZ:')
print(f'Rendimiento esperado: {opt_portfolio_expected_return*100}')
print(f'Varianza del portafolio: {opt_portfolio_var*100}')
print(f'Riesgo del portafolio: {opt_portfolio_risk*100}')

print('\n\nDel 100% de tu capital, el modelo sugiere\
invertir las siguientes proporciones en cada activo:')
display(cleaned_ratios*100)
     

PORTAFOLIO ÓPTIMO DE MARKOWITZ:
Rendimiento esperado: [0.09885579]
Varianza del portafolio: [[0.03085381]]
Riesgo del portafolio: 1.7565251996461473


Del 100% de tu capital, el modelo sugiereinvertir las siguientes proporciones en cada activo:


/tmp/ipykernel_30474/2863779071.py:62: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  opt_portfolio_risk = math.sqrt(opt_portfolio_var)


AAPL    28.108995
MSFT    42.232279
AMZN    16.041771
TSLA     0.000000
BABA    13.616955
dtype: float64